In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
module_path = os.path.abspath("/home/neurobook/Desktop/Research/_dev/conic-tools/") 
sys.path.insert(0, module_path)
module_path = os.path.abspath("/home/neurobook/Desktop/Research/_dev/PySpike/") 
sys.path.insert(0, module_path)

--- 
The following methods haven't been integrated in the tools yet... Have been adapted from [Schultz-Lab](https://github.com/schultzlab/Neural_Manifolds/)


In [ ]:
import numpy as np
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.manifold import Isomap, LocallyLinearEmbedding, SpectralEmbedding, TSNE
import umap

import NMLfunc as nml

from auxiliary import pairwise_distances, plot_embedding, intrinsic_dimensionality, linear_decoder, reconstruction_loss

import pickle
import warnings
warnings.filterwarnings("ignore")

In [ ]:
with open('../../data/states1_small.pkl', 'rb') as fp:
    states1 = pickle.load(fp)
# with open('../../data/states2.pkl', 'rb') as fp:
#     states2 = pickle.load(fp)
# with open('../../data/states3.pkl', 'rb') as fp:
#     states3 = pickle.load(fp)

### Pairwise distances
A good way to start is to look at the temporal structure of the population dynamics. 

In [ ]:
# obtain cosine distance matrix
down = 10 # downsampling factor for speeding up calculations (and memory constraints)
X = states1.matrix[:,::down].T

D = pairwise_distances(X, metric='cosine', save='./plots/time-structure.png')

In [ ]:
# reconstruction parameters
K_lle = 10
LAMBDA = 1

# global parameters
DIMS = 5
AXIS_LIM = 1

# Load data
ds_plt = 1 # downsample for plotting
down = 3 # downsampling param for manifold embedding

# initialise dicts to store decoding scores
DIM = {}
RMSE = {}; R = {}
rec_corr = {}
var_svd = {}; H = {}

In [ ]:
# Plotting parameters
sns.set(style='ticks',font_scale=2)
AXIS_LIM = 1
colb = ['#377eb8', '#ff7f00', '#4daf4a',
          '#f781bf', '#a65628', '#984ea3',
          '#999999', '#e41a1c', '#dede00']
plt.set_cmap('hsv') # circular cmap
# make svg text editable in illustrator
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

#### 1. Principal Component Analysis (PCA)

The simplest and most straightforward.

In [ ]:
### Fit embedding
EMBD = 'PCA'
pca = PCA(n_components=DIMS)
x_embd = pca.fit_transform(X)
x_embd = x_embd / np.max(np.abs(x_embd)) # normalise the values
AXIS_LIM = np.max(x_embd)

time_axis = np.arange(x_embd.shape[0], dtype=int)

In [ ]:
plot_embedding(x_embd, ds_plt, time_axis, AXIS_LIM, title="PCA", display=True, save='./plots/pca1.png')

In [ ]:
### Intrinsic dimensionality
nstep = 30
Nneigh, radii, p = intrinsic_dimensionality(x_embd, nstep=nstep, metric='euclidean', fit='std', thr_start=100, thr_fi=5e3, save='./plots/pca2-dim.png')
DIM[EMBD] = p[0]

In [ ]:
### decoding
rmse, r = linear_decoder(x_embd, DIMS, cv=10, labels=time_axis)
RMSE[EMBD] = rmse; R[EMBD] = r

# reconstruction
rec_loss = reconstruction_loss(X, x_embd, DIMS, K_lle, LAMBDA, cv=10, plot=True, display=True, save='./plots/{}-reconstruction_loss.png'.format(EMBD))
rec_corr[EMBD] = rec_loss

#### 2. Multi-Dimensional Scaling (MDS)

Like PCA, this is a linear mapping to find a (linear) projection of the high-dimensional data onto a low-dimensional sub-space.

In [ ]:
### Fit embedding
EMBD = 'MDS'
x_embd,eig_mds_dff = nml.cmdscale(D)
x_embd = x_embd / np.max(x_embd)

In [ ]:
plot_embedding(x_embd, ds_plt, time_axis, AXIS_LIM, title="MDS", display=True, save='./plots/mds1.png')

In [ ]:
### Dimensionality
nstep = 30
Nneigh, radii, p = intrinsic_dimensionality(x_embd[:,:DIMS], nstep=nstep, metric='euclidean', fit='std', thr_start=100, thr_fi=5e3, save='./plots/mds2.png')
DIM[EMBD] = p[0]

In [ ]:
# Reconstruction from embedding
rmse, r = linear_decoder(x_embd, DIMS, cv=10, labels=time_axis)
RMSE[EMBD] = rmse; R[EMBD] = r

rec_loss = reconstruction_loss(X, x_embd, DIMS, K_lle, LAMBDA, cv=10, plot=True, display=True, save='./plots/{}-reconstruction_loss.png'.format(EMBD))
rec_corr[EMBD] = rec_loss

#### 3. Isomap


In [ ]:
### Fit embedding
EMBD = 'Isomap'
COS = 1 # set to 1 for embedding the cosine distance matrix
if COS:
    isomap = Isomap(n_components=DIMS, n_neighbors=40)
    x_embd = isomap.fit_transform(D)
else:
    isomap = Isomap(n_components=DIMS, n_neighbors=40, metric='minkowski')
    x_embd = isomap.fit_transform(X)
x_embd = x_embd / np.max(x_embd)

In [ ]:
plot_embedding(x_embd, ds_plt, time_axis, AXIS_LIM, title=EMBD, display=True, save='./plots/{}1.png'.format(EMBD))

In [ ]:
### Dimensionality
nstep = 30
Nneigh, radii, p = intrinsic_dimensionality(x_embd[:,:DIMS], nstep=nstep, metric='euclidean', fit='std', thr_start=100, thr_fi=5e3, save='./plots/{}2.png'.format(EMBD))
DIM[EMBD] = p[0]

In [ ]:
# Reconstruction from embedding
rmse, r = linear_decoder(x_embd, DIMS, cv=10, labels=time_axis)
RMSE[EMBD] = rmse; R[EMBD] = r

rec_loss = reconstruction_loss(X, x_embd, DIMS, K_lle, LAMBDA, cv=10, plot=True, display=True, save='./plots/{}-reconstruction_loss.png'.format(EMBD))
rec_corr[EMBD] = rec_loss

#### 4. Locally Linear Embedding (LLE)

In [ ]:
### fit embedding
EMBD = 'LLE'
lle = LocallyLinearEmbedding(n_components=DIMS, n_neighbors=60, method='modified')
x_embd = lle.fit_transform(D)
x_embd = x_embd / np.max(x_embd)

In [ ]:
plot_embedding(x_embd, ds_plt, time_axis, AXIS_LIM, title=EMBD, display=True, save='./plots/{}1.png'.format(EMBD))

In [ ]:
### Dimensionality
nstep = 30
Nneigh, radii, p = intrinsic_dimensionality(x_embd[:,:DIMS], nstep=nstep, metric='euclidean', fit='std', thr_start=100, thr_fi=5e3, save='./plots/{}2.png'.format(EMBD))
DIM[EMBD] = p[0]

In [ ]:
# Reconstruction from embedding
rmse, r = linear_decoder(x_embd, DIMS, cv=10, labels=time_axis)
RMSE[EMBD] = rmse; R[EMBD] = r

rec_loss = reconstruction_loss(X, x_embd, DIMS, K_lle, LAMBDA, cv=10, plot=True, display=True, save='./plots/{}-reconstruction_loss.png'.format(EMBD))
rec_corr[EMBD] = rec_loss

#### 5. Laplacian Eigenmaps (Spectral Embedding)

In [ ]:
# fit embedding
EMBD = 'Spectral'
SE = SpectralEmbedding(n_components=DIMS, affinity='nearest_neighbors',
                       n_neighbors=60)
x_embd = SE.fit_transform(D)
x_embd = x_embd / np.max(x_embd)

In [ ]:
plot_embedding(x_embd, ds_plt, time_axis, AXIS_LIM, title=EMBD, display=True, save='./plots/{}1.png'.format(EMBD))

In [ ]:
### Dimensionality
nstep = 30
Nneigh, radii, p = intrinsic_dimensionality(x_embd[:,:DIMS], nstep=nstep, metric='euclidean', fit='std', thr_start=100, thr_fi=5e3, save='./plots/{}2.png'.format(EMBD))
DIM[EMBD] = p[0]

In [ ]:
# Reconstruction from embedding
rmse, r = linear_decoder(x_embd, DIMS, cv=10, labels=time_axis)
RMSE[EMBD] = rmse; R[EMBD] = r

rec_loss = reconstruction_loss(X, x_embd, DIMS, K_lle, LAMBDA, cv=10, plot=True, display=True, save='./plots/{}-reconstruction_loss.png'.format(EMBD))
rec_corr[EMBD] = rec_loss

#### 6. t-SNE

In [ ]:
# fit embedding
EMBD = 'tSNE'
SE = TSNE(n_components=3, metric='euclidean', perplexity=90, random_state=42)
x_embd = SE.fit_transform(X)
x_embd = x_embd / np.max(x_embd)

In [ ]:
plot_embedding(x_embd, ds_plt, time_axis, AXIS_LIM, title=EMBD, display=True, save='./plots/{}1.png'.format(EMBD))

In [ ]:
### Dimensionality
nstep = 30
Nneigh, radii, p = intrinsic_dimensionality(x_embd[:,:DIMS], nstep=nstep, metric='euclidean', fit='std', thr_start=100, thr_fi=5e3, save='./plots/{}2.png'.format(EMBD))
DIM[EMBD] = p[0]

In [ ]:
# Reconstruction from embedding
rmse, r = linear_decoder(x_embd, DIMS, cv=10, labels=time_axis)
RMSE[EMBD] = rmse; R[EMBD] = r

rec_loss = reconstruction_loss(X, x_embd, DIMS, K_lle, LAMBDA, cv=10, plot=True, display=True, save='./plots/{}-reconstruction_loss.png'.format(EMBD))
rec_corr[EMBD] = rec_loss

#### 7. UMAP

In [ ]:
### Fit embedding
EMBD = 'UMAP'
SE = umap.UMAP(n_components=DIMS, metric='cosine', n_neighbors=70, random_state=42)
x_embd = SE.fit_transform(X)
x_embd = nml.centre_scale(x_embd)

In [ ]:
plot_embedding(x_embd, ds_plt, time_axis, AXIS_LIM, title=EMBD, display=True, save='./plots/{}1.png'.format(EMBD))

In [ ]:
### Dimensionality
nstep = 30
Nneigh, radii, p = intrinsic_dimensionality(x_embd[:,:DIMS], nstep=nstep, metric='euclidean', fit='std', thr_start=100, thr_fi=5e3, save='./plots/{}2.png'.format(EMBD))
DIM[EMBD] = p[0]

In [ ]:
# Reconstruction from embedding
rmse, r = linear_decoder(x_embd, DIMS, cv=10, labels=time_axis)
RMSE[EMBD] = rmse; R[EMBD] = r

rec_loss = reconstruction_loss(X, x_embd, DIMS, K_lle, LAMBDA, cv=10, plot=True, display=True, save='./plots/{}-reconstruction_loss.png'.format(EMBD))
rec_corr[EMBD] = rec_loss

In [ ]:
DIM.keys()

---
## Comparisons

In [ ]:
plt.figure(figsize=(8,6))
LBL = 'real'
lbl = ['PCA', 'MDS', 'Isomap', 'LLE', 'Spectral', 'tSNE', 'UMAP']
for n,e in enumerate(lbl):
    plt.bar(n, DIM[e], width=.95, color=colb[n], label=e)
sns.despine()
# plt.hlines(1, 6.5, -0.5, color='k', linestyle='--')
plt.ylim(0, 3)
plt.ylabel('Dimensionality')
plt.xticks(range(7), lbl, rotation=45);
plt.savefig('./plots/dimensionality_'+LBL+'.png')

In [ ]:
sns.set(style='ticks', font_scale=2)
plt.figure(figsize=(10,6))
for n,k in enumerate(R):
    sem = stats.sem(RMSE[k],1, nan_policy='omit')
    plt.errorbar(range(1,DIMS+1), np.nanmean(RMSE[k],1), yerr=sem, label=k, color=colb[n], alpha=1)
plt.legend(frameon=False, loc=1)
plt.xticks(range(1,DIMS+1))
plt.ylabel('RMSE')
plt.xlabel('Number of dimensions')
sns.despine()
# saving
plt.savefig('./plots/decode_rmseErr_'+LBL+'.png')

plt.figure(figsize=(10,6))
for n,k in enumerate(R):
    sem = stats.sem(R[k],1, nan_policy='omit')
    plt.errorbar(range(1,DIMS+1), np.nanmean(R[k],1), yerr=sem, label=k, color=colb[n], alpha=1)
plt.ylim(0,1)
plt.xticks(range(1,DIMS+1))
plt.ylabel('Decoding performance [$r$]')
plt.xlabel('Number of dimensions')
plt.legend(frameon=False, loc=5)
sns.despine()
# saving
plt.savefig('./plots/decode_corrErr_'+LBL+'.png')

In [ ]:
# Reconstuction correlation
keys = list(rec_corr.keys())
plt.figure(figsize=(10,6))
for n,k in enumerate(keys):
    sem = stats.sem(rec_corr[k],1, nan_policy='omit')
    plt.errorbar(range(1,DIMS+1), np.nanmean(rec_corr[k],1), yerr=sem, label=k, color=colb[n], alpha=1)
# plt.ylim(0,.8)
plt.ylabel('Reconstruction similarity [$r$]')
plt.xlabel('Number of dimensions')
plt.xticks(range(1,DIMS+1))
sns.despine()
# saving
plt.savefig('./plots/rec_corr_'+LBL+'.png')

In [ ]:
RMSE['LLE'].mean()

In [ ]:
# Total error
plt.figure(figsize=(8,6))
LBL = 'real'
lbl = ['PCA', 'MDS', 'Isomap', 'LLE', 'Spectral', 'tSNE', 'UMAP']
for n,e in enumerate(lbl):
    plt.bar(n, RMSE[e].mean(), width=.95, color=colb[n], label=e)
sns.despine()
# plt.hlines(1, 6.5, -0.5, color='k', linestyle='--')
plt.ylim(2000, 2600)
plt.ylabel('Mean RMSE')
plt.xticks(range(7), lbl, rotation=45);
plt.savefig('./plots/totalRMSE_'+LBL+'.png')

In [ ]:
# Total reconstruction similarity
plt.figure(figsize=(8,6))
LBL = 'real'
lbl = ['PCA', 'MDS', 'Isomap', 'LLE', 'Spectral', 'tSNE', 'UMAP']
for n,e in enumerate(lbl):
    plt.bar(n, rec_corr[e].mean(), width=.95, color=colb[n], label=e)
sns.despine()
# plt.hlines(1, 6.5, -0.5, color='k', linestyle='--')
plt.ylim(0.5, 0.8)
plt.ylabel('Mean Reconstruction accuracy')
plt.xticks(range(7), lbl, rotation=45);
plt.savefig('./plots/totalRecAcc_'+LBL+'.png')

In [ ]:
# Total reconstruction similarity
plt.figure(figsize=(8,6))
LBL = 'real'
lbl = ['PCA', 'MDS', 'Isomap', 'LLE', 'Spectral', 'tSNE', 'UMAP']
for n,e in enumerate(lbl):
    plt.bar(n, R[e].mean(), width=.95, color=colb[n], label=e)
sns.despine()
# plt.hlines(1, 6.5, -0.5, color='k', linestyle='--')
# plt.ylim(0.5, 0.8)
plt.ylabel('Linear Decoder accuracy')
plt.xticks(range(7), lbl, rotation=45);
plt.savefig('./plots/totalDecAcc_'+LBL+'.png')